# Algoritmo de Otimização - Calendários de voos

### Imports

In [4]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose

### Representação do problema

In [6]:
pessoas: list[tuple[str, str]] = [
    ('Lisboa', 'LIS'),
    ('Madrid', 'MAD'),
    ('Paris', 'CDG'),
    ('Dublin', 'DUB'),
    ('Bruxelas', 'BRU'),
    ('Londres', 'LHR'),
]

In [7]:
destino = 'FCO'

In [8]:
voos = {}
with open('dataset/flights.txt') as f:
    for linha in f:
        origem, destino, saida, chegada, preco = linha.split(',')
        voos.setdefault((origem, destino), [])
        voos[(origem, destino)].append((saida, chegada, int(preco.replace('\n', ''))))

In [9]:
voos[('FCO', 'LIS')]

[('6:19', '8:13', 239),
 ('8:04', '10:59', 136),
 ('9:31', '11:43', 210),
 ('11:07', '13:24', 171),
 ('12:31', '14:02', 234),
 ('14:05', '15:47', 226),
 ('15:07', '17:21', 129),
 ('16:35', '18:56', 144),
 ('18:25', '20:34', 205),
 ('20:05', '21:44', 172)]

In [15]:
def imprimir_voos(agenda):
  id_voo = -1
  total_preco = 0
  for i in range(len(agenda) // 2):
    nome = pessoas[i][0]
    origem = pessoas[i][1]
    id_voo += 1
    ida = voos[(origem, destino)][agenda[id_voo]]
    total_preco += ida[2]
    id_voo += 1
    volta = voos[(destino, origem)][agenda[id_voo]]
    total_preco += volta[2]
    print('%10s%10s %5s-%5s %3s %5s-%5s %3s' % (nome, origem, ida[0], ida[1], ida[2],
                                                volta[0], volta[1], volta[2]))
  print('Preço total: ', total_preco)

In [30]:
agenda = [1,0, 3,2, 7,1, 6,3, 2,4, 5,3]
imprimir_voos(agenda)

    Lisboa       LIS  7:39-10:24 219  6:19- 8:13 239
    Madrid       MAD 11:01-12:39 260  9:11-10:42 172
     Paris       CDG 17:07-20:04 291  8:23-11:07 143
    Dublin       DUB 15:27-17:18 151 10:33-12:03  74
  Bruxelas       BRU  9:08-12:12 364 12:20-16:34 500
   Londres       LHR 13:40-15:38 137 10:32-13:16 139
Preço total:  2689


In [31]:
100**12

1000000000000000000000000

### Fitness Function

In [10]:
def fitness_function(agenda):
  id_voo = -1
  total_preco = 0
  for i in range(len(agenda) // 2):
    origem = pessoas[i][1]
    id_voo += 1
    ida = voos[(origem, destino)][agenda[id_voo]]
    total_preco += ida[2]
    id_voo += 1
    volta = voos[(destino, origem)][agenda[id_voo]]
    total_preco += volta[2]

  return total_preco

In [11]:
agenda = [1,7, 3,1, 1,1, 6,3, 2,4, 5,3]
fitness_function(agenda)

2619

In [12]:
fitness = mlrose.CustomFitness(fitness_function)
problema = mlrose.DiscreteOpt(length=12, fitness_fn=fitness, maximize=False, max_val=10)

### Hill Climb

In [13]:
melhor_solucao, melhor_custo = mlrose.hill_climb(problema)
melhor_solucao, melhor_custo

(array([2, 6, 5, 7, 8, 1, 3, 6, 0, 2, 9, 1], dtype=int32), 1566.0)

In [16]:
imprimir_voos(melhor_solucao)

    Lisboa       LIS  9:15-12:03  99 15:07-17:21 129
    Madrid       MAD 14:22-16:32 126 17:06-20:00  95
     Paris       CDG 18:23-21:35 134  8:23-11:07 143
    Dublin       DUB 11:16-13:29  83 15:25-16:58  62
  Bruxelas       BRU  6:12-10:22 230  9:49-13:51 229
   Londres       LHR 20:30-23:11 114  8:19-11:16 122
Preço total:  1566


In [17]:
voos[('LIS', 'FCO')]

[('6:11', '8:31', 249),
 ('7:39', '10:24', 219),
 ('9:15', '12:03', 99),
 ('11:08', '13:07', 175),
 ('12:18', '14:56', 172),
 ('13:37', '15:08', 250),
 ('15:03', '16:42', 135),
 ('16:51', '19:09', 147),
 ('18:12', '20:17', 242),
 ('20:05', '22:06', 261)]

### Simulated Annealing

In [55]:
from mlrose.decay import ExpDecay

melhor_solucao, melhor_custo = mlrose.simulated_annealing(problema, schedule=ExpDecay(), random_state=1)
melhor_solucao, melhor_custo

(array([7, 9, 5, 8, 8, 0, 1, 5, 3, 3, 9, 5], dtype=int32), 1865.0)

In [56]:
imprimir_voos(melhor_solucao)

    Lisboa       LIS 16:51-19:09 147 20:05-21:44 172
    Madrid       MAD 14:22-16:32 126 18:33-20:22 143
     Paris       CDG 18:23-21:35 134  6:33- 9:14 172
    Dublin       DUB  8:04-10:11  95 13:39-15:30  74
  Bruxelas       BRU 10:30-14:57 290 10:51-14:16 256
   Londres       LHR 20:30-23:11 114 13:37-15:33 142
Preço total:  1865


### Algoritmo Genético

In [63]:
melhor_solucao, melhor_custo = mlrose.genetic_alg(problem=problema, pop_size=500, mutation_prob=0.2, random_state=1)
melhor_solucao, melhor_custo

(array([7, 3, 1, 2, 9, 0, 3, 8, 0, 3, 4, 1]), 1956.0)